In [1]:
from AOC_utils import get_day
import numpy as np

day = 21
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 21 input already downloaded
5


['480A', '143A', '983A', '382A', '974A']

In [2]:
example = '''
029A
980A
179A
456A
379A
'''.split('\n')[1:-1]

In [3]:
from itertools import permutations

In [17]:
def solve_day(data):
    part1 = 0
    part2 = 0

    final_codes = data

    dir_map = {'<': np.array([0, -1]), '>': np.array([0, 1]), '^': np.array([-1, 0]), 'v': np.array([1, 0])}

    numpad = np.array([['7', '8', '9'], 
                       ['4', '5', '6'], 
                       ['1', '2', '3'],
                       [' ', '0', 'A']])
    
    robot_pad = np.array([[' ', '^', 'A'],
                         ['<', 'v', '>']])
    
    def sim_pad(start_loc, direction):
        new_loc = start_loc.copy()
        new_loc += dir_map[direction]
        return numpad[tuple(new_loc)]
    
    numpad_mapping = {}
    for current_n in numpad.flatten():
        for target_n in numpad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            current_loc = np.argwhere(numpad == current_n)[0]
            target_loc = np.argwhere(numpad == target_n)[0]

            distance = current_loc - target_loc
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            # if current_n == 'A' or current_n == '0':
            #     move = (up * '^') + (down * 'v') + (left * '<') + (right * '>')
            # else:
            move = (left * '<') + (right * '>') + (up * '^') + (down * 'v')

            if move == '':
                numpad_mapping[current_n + target_n] = ['']
                continue

            moves = np.unique([''.join(x) for x in list(permutations(list(move)))], axis=0)

            good_moves = []
            for move in moves:
                loc = current_loc.copy()
                valid = True
                for direction in move:
                    if sim_pad(loc, direction) == ' ':
                        valid = False
                        break
                    loc += dir_map[direction]
                if valid:
                    good_moves.append(move)
            moves = good_moves

            numpad_mapping[current_n + target_n] = moves

    robot_mapping = {}
    for current_n in robot_pad.flatten():
        for target_n in robot_pad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            distance = np.argwhere(robot_pad == current_n)[0] - np.argwhere(robot_pad == target_n)[0]
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            # if current_n == '<':
            #     move = (left * '<') + (right * '>') + (up * '^') + (down * 'v')
            # else:
            move = (up * '^') + (down * 'v') + (left * '<') + (right * '>')

            if move == '':
                robot_mapping[current_n + target_n] = ['']
                continue

            moves = np.unique([''.join(x) for x in list(permutations(list(move)))], axis=0).tolist()

            good_moves = []
            for move in moves:
                loc = np.argwhere(robot_pad == current_n)[0]
                valid = True
                for direction in move:
                    if sim_pad(loc, direction) == ' ':
                        valid = False
                        break
                    loc += dir_map[direction]
                if valid:
                    good_moves.append(move)
            moves = good_moves

            robot_mapping[current_n + target_n] = moves

    def fcode_to_robot(c, current='A'):
        return [x + 'A' for x in numpad_mapping[current + c]]
    
    def robot_to_robot(code, current='A'):
        moves_so_far = ['']
        for c in code:
            new_moves_so_far = []
            new_possabilities = [x + 'A' for x in robot_mapping[current + c]]
            for move in moves_so_far:
                for new_possability in new_possabilities:
                    new_moves_so_far.append(move + new_possability)
            moves_so_far = new_moves_so_far
            current = c
        return moves_so_far
    

    for code in final_codes:
        print(code)
        final_code = ''
        current = 'A'
        for ch in code:
            numpad_code_possabilities = fcode_to_robot(ch, current)
            # print(numpad_code)

            robot_code_possabilities = np.concatenate([np.concatenate([robot_to_robot(r) for r in robot_to_robot(n)]) for n in numpad_code_possabilities])
            # print(robot_code)
            # robot_code2 = np.concatenate([robot_to_robot(n) for n in robot_code_possabilities])
            robot_code2 = robot_code_possabilities

            # get shortest
            robot_code2 = sorted(robot_code2, key=lambda x: len(x))
            # print(robot_code2)
            robot_code2 = robot_code2[0]
            print(robot_code2)
            final_code += robot_code2
            current = ch

        # print(robot_code2)
        # print(len(robot_code2), int(code[:-1]))
        print(final_code)
        part1 += len(final_code) * int(code[:-1])

    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example)

029A
<<vAA>A>^AvAA<^A>A
<<vA>>^AvA^A
<vA>^A<<vA>^A>AAvA^A
<<vA>A>^AAAvA<^A>A
<<vAA>A>^AvAA<^A>A<<vA>>^AvA^A<vA>^A<<vA>^A>AAvA^A<<vA>A>^AAAvA<^A>A
980A
<<vA>>^AAAvA^A
<<vAA>A>^AvAA<^A>A
<<vA>A>^AAAvA<^A>A
<vA>^A<A>A
<<vA>>^AAAvA^A<<vAA>A>^AvAA<^A>A<<vA>A>^AAAvA<^A>A<vA>^A<A>A
179A
<<vA>>^A<<vA>A>^AAvAA<^A>A
<<vA>>^AAvA^A
<vA>^AA<A>A
<<vA>A>^AAAvA<^A>A
<<vA>>^A<<vA>A>^AAvAA<^A>A<<vA>>^AAvA^A<vA>^AA<A>A<<vA>A>^AAAvA<^A>A
456A
<<vA>>^AA<<vA>A>^AAvAA<^A>A
<vA>^A<A>A
<vA>^A<A>A
<<vA>A>^AAvA<^A>A
<<vA>>^AA<<vA>A>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<<vA>A>^AAvA<^A>A
379A
<<vA>>^AvA^A
<<vAA>A>^AAvA<^A>AAvA^A
<vA>^AA<A>A
<<vA>A>^AAAvA<^A>A
<<vA>>^AvA^A<<vAA>A>^AAvA<^A>AAvA^A<vA>^AA<A>A<<vA>A>^AAAvA<^A>A
part 1: 126384
part 2: 0


In [18]:
solve_day(input_data)

480A
<<vA>>^AA<<vA>A>^AAvAA<^A>A
<vA>^A<<vA>^A>AvA^A
<<vA>A>^AAAvA<^A>A
<vA>^A<A>A
<<vA>>^AA<<vA>A>^AAvAA<^A>A<vA>^A<<vA>^A>AvA^A<<vA>A>^AAAvA<^A>A<vA>^A<A>A
143A
<<vA>>^A<<vA>A>^AAvAA<^A>A
<<vA>>^AvA^A
<<vA>A>^AvA^AA<A>A
<<vA>A>^AvA<^A>A
<<vA>>^A<<vA>A>^AAvAA<^A>A<<vA>>^AvA^A<<vA>A>^AvA^AA<A>A<<vA>A>^AvA<^A>A
983A
<<vA>>^AAAvA^A
<<vAA>A>^AvAA<^A>A
<<vA>A>^AAvA^A<A>A
<<vA>A>^AvA<^A>A
<<vA>>^AAAvA^A<<vAA>A>^AvAA<^A>A<<vA>A>^AAvA^A<A>A<<vA>A>^AvA<^A>A
382A
<<vA>>^AvA^A
<<vAA>A>^AvA<^A>AAvA^A
<<vA>A>^AAvA<^A>A
<<vA>A>^AvA^A<A>A
<<vA>>^AvA^A<<vAA>A>^AvA<^A>AAvA^A<<vA>A>^AAvA<^A>A<<vA>A>^AvA^A<A>A
974A
<<vA>>^AAAvA^A
<<vAA>A>^AAvAA<^A>A
<<vA>A>^AvA<^A>A
<vA>^AA<<vA>>^AAvA<^A>A
<<vA>>^AAAvA^A<<vAA>A>^AAvAA<^A>A<<vA>A>^AvA<^A>A<vA>^AA<<vA>>^AAvA<^A>A
part 1: 206798
part 2: 0


In [6]:
206798